# Data prep for image recognition

The purpose of this short notebook is to introduce the most basic features of the OpenCV library, focusing on features that will make it possible to use intelligent APIs on image data.

In [ ]:
import cv2
import numpy as np

The first thing we'll try is reading an image from a file.  OpenCV makes it easy to decode popular image formats, and this notebook has access to an image file we can read.

In [ ]:
img = cv2.imread("otto.jpg", cv2.IMREAD_COLOR)

## Working with images as arrays

This will get us a `numpy` array containing the pixels from a picture of a confused schnauzer who did not expect to wind up unable to get out of the clothes basket. 

We can look at the size of the array:

In [ ]:
img.shape

We can examine the image itself by plotting it.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.imshow(img)

While our focus is on using pretrained models, if you're training a model, it may be useful to transform, blur, or resize images in order to generate more training data from a few images.  Since our images are `numpy` arrays, this is relatively straightforward in general, but OpenCV provides functions to make these tasks even easier.  We'll see how to

- blur an input image with a 15x15 box blur,
- resize an image and interpolate between pixels in the source data, and
- rotate an image without calculating a transformation matrix

First, let's look at box blur:

In [ ]:
plt.imshow(cv2.blur(img, (15,15)))

We can also scale the image by a factor of 3 on both axes (notice the difference in the axes on the plotted image, even though the size doesn't change).

In [ ]:
plt.imshow(cv2.resize(img, None, fx=3, fy=3, interpolation=cv2.INTER_CUBIC))

It's also possible to stretch the image by scaling along axes differently:

In [ ]:
plt.imshow(cv2.resize(img, None, fx=2.5, fy=3, interpolation=cv2.INTER_CUBIC))

We can also rotate the image.  Recall that rotation is an affine tranformation on image matrices.  OpenCV provides a function to calculate the transformation matrix, given a point to rotate around, an angle of rotation, and a scaling factor.  Here we'll rotate the image around its center by 15 degrees while scaling by 1.3x.

In [ ]:
rows, cols, _ = img.shape
center = (cols / 2, rows / 2)
angle = 15 # degrees
scale = 1.3
rotationMatrix = cv2.getRotationMatrix2D(center, angle, scale)
plt.imshow(cv2.warpAffine(img, rotationMatrix, (cols, rows)))

## Working with image data in byte arrays

In many non-batch applications, we won't be actually processing _files_; instead, we'll be dealing with binary data, whether passed as a base64-encoded string to a HTTP request or stored in a blob as part of structured data on a stream.  OpenCV is able to decode this raw binary data just as it is able to decode files; this last part of the notebook will show you how to do it.

We'll start by getting a Python `bytearray` with the contents of a file.  Notice that, while we have a JPEG file, we aren't storing the file type anywhere.

In [ ]:
with open("otto.jpg", "rb") as f:
    img_bytes = bytearray(f.read())

Now that we have a `bytearray` of the file's contents, we'll convert that into a flat NumPy array:

In [ ]:
imgarr = np.asarray(img_bytes, dtype=np.uint8)
imgarr

The OpenCV `imdecode` function will inspect this flat array and parse it as an image, inferring the right type and dimensions and returning a multidimensional array with an appropriate shape.

In [ ]:
img2 = cv2.imdecode(imgarr, cv2.IMREAD_COLOR)

We then have a multidimensional array that we can use just as we did the image we read from a file.

In [ ]:
plt.imshow(img2)

## Image intensities

We can also plot histograms for each channel of the image.  (This example code is taken from the [OpenCV documentation](https://docs.opencv.org/3.1.0/d1/db7/tutorial_py_histogram_begins.html).)  You can see that the image of the dog is underexposed.

In [ ]:
for i, color in enumerate(["b", "g", "r"]):
    histogram = cv2.calcHist([img], [i], None, [256], [0, 256])
    plt.plot(histogram, color=color)
    plt.xlim([0, 256])
plt.show()

# Object detection with pretrained models

Now that we've seen how to use some of the basic capabilities of OpenCV to parse image data into a matrix of pixels -- and then to perform useful image transformations and analyses on this matrix -- we're ready to see how to use a pretrained model to identify objects in real images.

We'll use a pretrained [YOLO](https://pjreddie.com/darknet/yolo/) ("you only look once") model and we'll load and score that model with the [darkflow](https://github.com/thtrieu/darkflow/) library, which is built on TensorFlow.

One of the key themes of this workshop is that you don't need a deep understanding of the techniques behind off-the-shelf models for language processing or image recognition in order to make use of them in your applications, but YOLO is a cool technique, so if you want to learn more about it, here's where to get started:

- [this paper](https://pjreddie.com/media/files/papers/yolo_1.pdf) explains the first version of YOLO and the basic technique,
- [this presentation](https://www.youtube.com/watch?v=NM6lrxy0bxs) presents the basics of the paper in a thirteen-minute video, and
- [this paper](http://homepages.inf.ed.ac.uk/ckiw/postscript/ijcv_voc09.pdf) provides a deeper dive into object detection (including some details on the mAP metric for evaluating classifier quality)

YOLO is so-called because previous object-detection techniques repeatedly ran image classifiers on multiple overlapping windows of an image; by contrast, YOLO "only looks once," identifying image regions that might contain an interesting object and then identifying which objects those regions might contain in a single pass.  It can be much faster than classic approaches; indeed, it can run in real time or faster with GPU acceleration.

## Loading our model

We'll start by loading a pretrained model architecture and model weights from files:

In [ ]:
from darkflow.net.build import TFNet
options = {"model": "cfg/yolo.cfg", "load": "/data/yolo.weights", "threshold" : 0.1}
yolo = TFNet(options)


Our next step is to use the model to identify some objects in an image.  We'll start with the dog image.  The `return_predict` method will return a list of predictions, each with a visual object class, a confidence score, and a bounding box.

In [ ]:
predictions = yolo.return_predict(img)
predictions

To be fair, most dogs spend a lot of time on sofas.

It is often useful to visualize what parts of the image were identified as objects.  We can use OpenCV to annotate the bounding boxes of each identified object in the image with the `cv2.rectangle` function.  Since this is destructive, we'll work on a copy of the image.

In [ ]:
def annotate(img, predictions, thickness=None):
    """ Copies the supplied image and annotates it with the bounding
        boxes of each identified object """
    annotated_img = np.copy(img)
    
    if thickness is None:
        thickness = max(img.shape(0), img.shape(1)) / 100
    
    for prediction in predictions:
        tl = prediction["topleft"]
        topleft = (tl["x"], tl["y"])
        br = prediction["bottomright"]
        bottomright = (br["x"], br["y"])
        # draw a white rectangle around the identified object
        white = (255,255,255)
        cv2.rectangle(annotated_img, topleft, bottomright, color=white, thickness=thickness)

    return annotated_img

In [ ]:
plt.imshow(annotate(img, predictions))

## Trying it out with other images

We can try this technique out with other images as well.

In [ ]:
from ipywidgets import interact
from os import listdir

def predict(imageFile):
    image = cv2.imread("/data/images/" + imageFile, cv2.IMREAD_COLOR)
    predictions = yolo.return_predict(image)
    plt.imshow(annotate(image, predictions, thickness=5))
    return predictions

interact(predict, imageFile = listdir("/data/images/"))

# Training custom models

It's outside the scope of this workshop (both in terms of time and content), but you've actually learned a lot of skills in this notebook that are applicable to training custom object detection models (e.g., to identify new kinds of objects).  Here's how you can get started.

1.  You'll need some labeled data; for object detection, this is going to be image files annotated with the bounding boxes and object classes of real-world object pictured in those images.  Good places to start are the [Pascal VOC 2012 dataset](http://host.robots.ox.ac.uk/pascal/VOC/voc2012/index.html#data) or the [COCO dataset](http://cocodataset.org/).
2.  If you want to identify new object classes (for example, a corporate logo), you'll need to add labeled images that contain these object classes.  Since you may not have many example images for the new object classes, you may want to generate synthetic images to augment your training set; there are [many approaches ranging from rotating and scaling input data to using neural networks to generate new examples](https://arxiv.org/pdf/1712.04621.pdf).  You already know how to transform and rotate images, of course!
3.  Actually training the model will depend on what framework you ultimately want to use for the project; [here are the instructions for Darkflow](https://github.com/thtrieu/darkflow#training-on-your-own-dataset).